In [9]:
import pandas as pd
import gensim

In [2]:
df = pd.read_csv("amazon_reviews.csv")

In [3]:
df.head(3)

,Unnamed: 0,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,0528881469,"[0, 0]",5.0,We got this GPS for my husband who is an (OTR)...,"06 2, 2013",AO94DHGC771SJ,amazdnu,Gotta have GPS!,1.370131e+09
1,1,0528881469,"[12, 15]",1.0,"I'm a professional OTR truck driver, and I bou...","11 25, 2010",AMO214LNFCEI4,Amazon Customer,Very Disappointed,1.290643e+09
2,2,0528881469,"[43, 45]",3.0,"Well, what can I say. I've had this unit in m...","09 9, 2010",A3N7T0DY83Y4IG,C. A. Freeman,1st impression,1.283990e+09


In [4]:
df.shape

(999, 10)

In [11]:
df.reviewText[5]
df['reviewText'] = df['reviewText'].fillna('').astype(str)

In [12]:
review_text = df.reviewText.apply(gensim.utils.simple_preprocess)
review_text #This will remove special characters like ('!','\', '?' etc)
#And also it will make the word to small letters and make it in an array of words of one review. So we make it simpler for word2vec. 

0      [we, got, this, gps, for, my, husband, who, is...
1      [professional, otr, truck, driver, and, bought...
2      [well, what, can, say, ve, had, this, unit, in...
3      [not, going, to, write, long, review, even, th...
4      [ve, had, mine, for, year, and, here, what, we...
                             ...                        
994    [cable, runs, lifetime, excellent, product, th...
995    [quality, of, connection, and, video, seems, g...
996    [what, is, there, to, say, needed, cable, to, ...
997    [can, find, nothing, wrong, with, this, cable,...
998    [works, perfectly, at, hd, and, in, all, other...
Name: reviewText, Length: 999, dtype: object

In [24]:
#now we are making a model which will predict the next word
#for example: king _____ his soldiers. The missing word is ordered. we learn it through srounding words
model = gensim.models.Word2Vec( #window can be 3 5 6 any number. NOT MORE THAN SENTENCE WORDS
    window = 5, #Window means the words before and after the target word. it take 10 words of the sentence before the target word and also it take after
    min_count = 2, #for this minimum words in sentence should be two.
    workers = 2 #it means how many CPU threads do you want for training
)

In [25]:
model.build_vocab(review_text, progress_per=50) #so it shows progress after each 100 words.

In [15]:
model.epochs

5

In [17]:
model.corpus_count #we have 999 total reviews 

999

In [26]:
model.train(review_text, total_examples=model.corpus_count, epochs= 15)

(1483509, 2065935)

In [ ]:
#we should have to save the model for NLP use in after works
model.save("your_path_where_to_save")

In [27]:
model.wv.most_similar("bad") # so it will shows the words that are similar to bad

[('deal', 0.9292589426040649),
 ('worth', 0.9160486459732056),
 ('otherwise', 0.9049379229545593),
 ('breaker', 0.89898282289505),
 ('certainly', 0.898230791091919),
 ('disappointed', 0.8929365873336792),
 ('length', 0.8925901055335999),
 ('buster', 0.8895811438560486),
 ('unacceptable', 0.881739616394043),
 ('learned', 0.8794098496437073)]

In [21]:
model.wv.most_similar("good") 

[('great', 0.9987720251083374),
 ('very', 0.9980811476707458),
 ('well', 0.9979926347732544),
 ('is', 0.9976755976676941),
 ('price', 0.9974967241287231),
 ('as', 0.9973298907279968),
 ('works', 0.9973104000091553),
 ('beautiful', 0.993677020072937),
 ('product', 0.9936079382896423),
 ('touch', 0.9931375980377197)]

In [28]:
#similarity score
model.wv.similarity(w1='cheap', w2='inexpensive')

0.83587766

In [29]:
model.wv.similarity(w1='great', w2='good') 

0.83909255

In [34]:
model.wv.similarity(w1='great', w2='nothing') 

1.0

In [35]:
model.wv.similarity(w1='great', w2='great') 

1.0

In [40]:
model.wv.similarity(w1='great', w2='tv')  #working fine but not so correct

0.25724843